In [1]:
import tensorflow as tf

print("TensorFlow version:", tf.__version__)
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))
print("GPU Details:", tf.config.list_physical_devices('GPU'))


TensorFlow version: 2.18.0
Num GPUs Available: 0
GPU Details: []


In [2]:
from keras.utils import to_categorical
from keras_preprocessing.image import load_img
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
import os
import pandas as pd
import numpy as np

In [3]:
TRAIN_DIR = "C:/Users/Asus/OneDrive/Documents/face recognition project/images/train"
TEST_DIR = "C:/Users/Asus/OneDrive/Documents/face recognition project/images/test"

In [4]:
def createdataframe(dir):
    image_paths = []
    labels = []
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir,label)):
            image_paths.append(os.path.join(dir,label,imagename))
            labels.append(label)
        print(label, "completed")
    return image_paths,labels

In [5]:

train = pd.DataFrame()
train['image'], train['label'] = createdataframe(TRAIN_DIR)

angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed


In [6]:
print(train)

                                                   image     label
0      C:/Users/Asus/OneDrive/Documents/face recognit...     angry
1      C:/Users/Asus/OneDrive/Documents/face recognit...     angry
2      C:/Users/Asus/OneDrive/Documents/face recognit...     angry
3      C:/Users/Asus/OneDrive/Documents/face recognit...     angry
4      C:/Users/Asus/OneDrive/Documents/face recognit...     angry
...                                                  ...       ...
28816  C:/Users/Asus/OneDrive/Documents/face recognit...  surprise
28817  C:/Users/Asus/OneDrive/Documents/face recognit...  surprise
28818  C:/Users/Asus/OneDrive/Documents/face recognit...  surprise
28819  C:/Users/Asus/OneDrive/Documents/face recognit...  surprise
28820  C:/Users/Asus/OneDrive/Documents/face recognit...  surprise

[28821 rows x 2 columns]


In [7]:

test = pd.DataFrame()
test['image'], test['label'] = createdataframe(TEST_DIR)

angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed


In [8]:
print(test)
print(test['image'])

                                                  image     label
0     C:/Users/Asus/OneDrive/Documents/face recognit...     angry
1     C:/Users/Asus/OneDrive/Documents/face recognit...     angry
2     C:/Users/Asus/OneDrive/Documents/face recognit...     angry
3     C:/Users/Asus/OneDrive/Documents/face recognit...     angry
4     C:/Users/Asus/OneDrive/Documents/face recognit...     angry
...                                                 ...       ...
7061  C:/Users/Asus/OneDrive/Documents/face recognit...  surprise
7062  C:/Users/Asus/OneDrive/Documents/face recognit...  surprise
7063  C:/Users/Asus/OneDrive/Documents/face recognit...  surprise
7064  C:/Users/Asus/OneDrive/Documents/face recognit...  surprise
7065  C:/Users/Asus/OneDrive/Documents/face recognit...  surprise

[7066 rows x 2 columns]
0       C:/Users/Asus/OneDrive/Documents/face recognit...
1       C:/Users/Asus/OneDrive/Documents/face recognit...
2       C:/Users/Asus/OneDrive/Documents/face recognit...
3       C

In [9]:
from tqdm.notebook import tqdm

In [10]:

def extract_features(images):
    features = []
    for image in tqdm(images):
        img = load_img(image,grayscale =  True )
        img = np.array(img)
        features.append(img)
    features = np.array(features)
    features = features.reshape(len(features),48,48,1)
    return features
    


In [ ]:
train_features = extract_features(train['image']) 

  0%|          | 0/28821 [00:00<?, ?it/s]

C:\Users\Asus\anaconda3\Lib\site-packages\keras_preprocessing\image\utils.py:107: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '


In [ ]:
test_features = extract_features(test['image'])

In [ ]:

x_train = train_features/255.0
x_test = test_features/255.0

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
# Step 1: Encode labels as integers
le = LabelEncoder()
y_train = le.fit_transform(train['label'])
y_test = le.transform(test['label'])

# Step 2: Convert to one-hot encoding (only once!)
y_train = to_categorical(y_train, num_classes=7)
y_test = to_categorical(y_test, num_classes=7)

# Optional: Check shape
print("y_train shape:", y_train.shape)  # Should be (num_samples, 7)
print("y_test shape:", y_test.shape)


In [ ]:
from keras.models import Sequential
from keras.layers import Input, Conv2D, MaxPooling2D, Dropout, Flatten, Dense

model = Sequential()

# 👇 Input Layer
model.add(Input(shape=(48, 48, 1)))

# 👇 Convolutional Layers
model.add(Conv2D(128, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

# 👇 Fully Connected Layers
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))

# 👇 Output Layer
model.add(Dense(7, activation='softmax'))


In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
model.fit(x=x_train, y=y_train, batch_size=128, epochs=100, validation_data=(x_test, y_test))




In [ ]:
model_json = model.to_json()
with open("emotiondetector.json",'w') as json_file:
    json_file.write(model_json)
model.save("emotiondetector.h5")

In [ ]:
from keras.models import model_from_json

In [ ]:
json_file = open("emotiondetector.json", "r")
model_json = json_file.read()
json_file.close()
model = model_from_json(model_json)
model.load_weights("emotiondetector.h5")

In [ ]:
label = ['angry','disgust','fear','happy','neutral','sad','surprise']

In [ ]:
def ef(image):
    img = load_img(image,grayscale =  True )
    feature = np.array(img)
    feature = feature.reshape(1,48,48,1)
    return feature/255.0

In [ ]:
image = 'images/train/sad/42.jpg'
print("original image is of sad")
img = ef(image)
pred = model.predict(img)
pred_label = label[pred.argmax()]
print("model prediction is ",pred_label)